In [ ]:
# Importing get_ipython function from IPython module.
# This allows access to the active interactive shell instance.
from IPython import get_ipython

# Importing the display function from IPython.display module.
# This is used to display rich representations of objects (e.g., dataframes, images) in Jupyter notebooks.
from IPython.display import display

In [ ]:
# Importing deque from collections module.
# deque (double-ended queue) is used for fast appends and pops from both ends.
from collections import deque

# Importing OpenCV for computer vision tasks such as image and video processing.
import cv2

# Importing NumPy for numerical operations, especially on arrays and matrices.
import numpy as np

# Importing the supervision library, which provides tools for computer vision tasks such as annotation and visualization.
import supervision as sv

In [ ]:
class BallAnnotator:
    """
    A class to annotate frames with circles of varying radii and colors.

    Attributes:
        radius (int): The maximum radius of the circles to be drawn.
        buffer (deque): A deque buffer to store recent coordinates for annotation.
        color_palette (sv.ColorPalette): A color palette for the circles.
        thickness (int): The thickness of the circle borders.
    """

    def __init__(self, radius: int, buffer_size: int = 5, thickness: int = 2):
        # Create a color palette using the 'jet' colormap with the given buffer size
        self.color_palette = sv.ColorPalette.from_matplotlib('jet', buffer_size)
        # Initialize a deque to store past detection coordinates with a fixed maximum length
        self.buffer = deque(maxlen=buffer_size)
        # Set the maximum radius for the circles
        self.radius = radius
        # Set the thickness of the circle borders
        self.thickness = thickness

    def interpolate_radius(self, i: int, max_i: int) -> int:
        """
        Interpolates the radius between 1 and the maximum radius based on the index.

        Args:
            i (int): The current index in the buffer.
            max_i (int): The maximum index in the buffer.

        Returns:
            int: The interpolated radius.
        """
        if max_i == 1:
            # If only one item, return max radius directly
            return self.radius
        # Linearly interpolate radius based on index
        return int(1 + i * (self.radius - 1) / (max_i - 1))

    def annotate(self, frame: np.ndarray, detections: sv.Detections) -> np.ndarray:
        """
        Annotates the frame with circles based on detections.

        Args:
            frame (np.ndarray): The frame to annotate.
            detections (sv.Detections): The detections containing coordinates.

        Returns:
            np.ndarray: The annotated frame.
        """
        # Extract bottom-center anchor points from detection bounding boxes
        xy = detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER).astype(int)
        # Append current detection coordinates to buffer
        self.buffer.append(xy)

        # Iterate through buffered coordinates to draw circles
        for i, xy in enumerate(self.buffer):
            # Get color for current index from the palette
            color = self.color_palette.by_idx(i)
            # Get interpolated radius for visual fading effect
            interpolated_radius = self.interpolate_radius(i, len(self.buffer))

            # Draw circles for all detection points in the current buffer entry
            for center in xy:
                frame = cv2.circle(
                    img=frame,
                    center=tuple(center),
                    radius=interpolated_radius,
                    color=color.as_bgr(),  # Convert color to BGR format for OpenCV
                    thickness=self.thickness
                )
        return frame

In [ ]:
class BallTracker:
    """
    A class used to track a football ball's position across video frames.

    The BallTracker class maintains a buffer of recent ball positions and uses this
    buffer to predict the ball's position in the current frame by selecting the
    detection closest to the average position (centroid) of the recent positions.

    Attributes:
        buffer (collections.deque): A deque buffer to store recent ball positions.
    """

    def __init__(self, buffer_size: int = 10):
        # Initialize a deque to hold recent ball position arrays, limited to buffer_size.
        self.buffer = deque(maxlen=buffer_size)

    def update(self, detections: sv.Detections) -> sv.Detections:
        """
        Updates the buffer with new detections and returns the detection closest to the
        centroid of recent positions.

        Args:
            detections (sv.Detections): The current frame's ball detections.

        Returns:
            sv.Detections: The detection closest to the centroid of recent positions.
                           If there are no detections, returns the input detections.
        """
        # Extract center anchor points from the detections (as (x, y) coordinates).
        xy = detections.get_anchors_coordinates(sv.Position.CENTER)

        # Add the current frame's detection coordinates to the buffer.
        self.buffer.append(xy)

        # If there are no detections in this frame, return the original detections as-is.
        if len(detections) == 0:
            return detections

        # Calculate the centroid (average position) of all coordinates in the buffer.
        centroid = np.mean(np.concatenate(self.buffer), axis=0)

        # Compute Euclidean distances from each current detection to the centroid.
        distances = np.linalg.norm(xy - centroid, axis=1)

        # Find the index of the detection closest to the centroid.
        index = np.argmin(distances)

        # Return only the detection that is closest to the centroid.
        return detections[[index]]